In [1]:
%matplotlib inline
import pandas as pd, ipywidgets as widgets, numpy as np, matplotlib.pyplot as plt, io
import string, math
from ipywidgets import interact, interactive, fixed, interact_manual

#Standard functions

#Function that ensures a given antibody platemap is formatted correctly (by columns, not rows)
#Mutable function that changes the data frame passed in by argument "ab_sheet"
#ie A1->B1-->H1->A2-->B2-->A12-->H12
#First, separate the Well ID into separate "Plate Number","Format Row" and "Format Column" values 
#(Row is Letter, Column is Number)
#Sort the platemap into proper Plate -> Column -> Row format
#Returns nothing, mutates the ab_sheet antibody plate map
def format_platemap(ab_sheet):
    ab_sheet["Format Row"] = ab_sheet["Well ID"].str.slice(start = 0, stop = 1)
    numeric_col = ab_sheet["Well ID"].str.slice(start = 1)
    numeric_col = pd.to_numeric(numeric_col)
    ab_sheet["Format Column"] = numeric_col
    ab_sheet.sort_values(by=["Plate Number","Format Column", "Format Row"], inplace=True, ignore_index = True)
    return

#Average function that takes a series with n=1, n=2 replicates in sequential order within the series, and 
#Separates n=1 and n=2 replicates resets indeces, and calculates average
#Returns a series with the averaged values, removing the replicate wells
def average(series_to_average):
    firsts = series_to_average.iloc[::2]
    seconds = series_to_average.iloc[1::2]
    firsts.reset_index(inplace = True, drop = True)
    seconds.reset_index(inplace = True, drop = True)
    averaged_series = firsts + seconds
    averaged_series = averaged_series/2
    return averaged_series

#Takes a Pandas dataframe as an input. Takes every column from the dataframe and concatenates them into one single series
#Returns the new series that has all of the data in one column, starting from 1st column -> max # of columns
def columnize(df_to_columnize):
    return_column = pd.Series()
    for column in df_to_columnize.columns:
        colToAdd = df_to_columnize[column]
        return_column = pd.concat([return_column, colToAdd], axis = 0, ignore_index = True)
    return return_column

In [19]:
#New functions written for M5 analysis

#Average function that returns an average series but also a series of the standard deviations for each average
def average_stddev(series_to_average):
    firsts = series_to_average.iloc[::2]
    seconds = series_to_average.iloc[1::2]
    firsts.reset_index(inplace = True, drop = True)
    seconds.reset_index(inplace = True, drop = True)
    averaged_series = firsts + seconds
    averaged_series = averaged_series/2
    stddev = (((firsts - averaged_series) ** 2) + ((seconds - averaged_series) ** 2) / 2) ** .5
    return averaged_series, stddev

#Parse data from the xlsx format from M5. Splits the raw data file into plates, not analysis ready, 
#and return a dictionary of {plate number : uncleaned data} key value pairs
#Accepts a Pandas dataframe that is the raw data from the M5, number of blocks in the raw data, 
#and the blocksize (default = 20)
def parse(data, numblocks, blocksize = 20):
    parsed_dict = {}
    for i in range(numblocks):
        rowstart = 0 + blocksize * i
        rowend = 18 + blocksize * i
        df = data.iloc[rowstart:rowend, 0:26]
        df.reset_index(inplace = True, drop = True)
        parsed_dict[i+1] = df
    return parsed_dict
        
#For each plate parsed, clean unnecessary data and return a dictionary of {plate name: cleaned data} key value pairs
#Accepts a dictionary of raw plates. Cleans each plate and renames axes to proper 384 well format
#Cleaning the plate means reformatting the data into a pandas DataFrame with proper multi-indexing: 8 Rows with n = 1/2
#for each row. 
#Returns a dictionary of cleaned plates
def clean(parsedplates):    
    cleaned_dict = {}
    for key, plate in parsedplates.items():
        platename = plate[1][0]
        data = plate.iloc[2:, 2:27].reset_index(drop = True)

        rows = list(range(1, 9))
        ns = [1, 2]
        multidex_array = [rows, ns]
        multidex = pd.MultiIndex.from_product(multidex_array, names = ["Row", "n"])
        
        cleaned = pd.DataFrame(data.values, index = multidex)
        cleaned.rename(columns = dict(zip(list(cleaned.columns), list(range(1, 25)))), inplace = True)
        cleaned.rename_axis("Column", axis = "columns", inplace = True)
        cleaned_dict[platename] = cleaned
    return cleaned_dict

#Function to divide one plate by another. Check keys. Plate name should be organized in 
#"*Target* *Assay*"" format, with assay being the last substring after a space. 
# platedic - a dictionary of {Plate Name: Data} key/value pairs to perform the division on
# assaynum - a string of the name of the assay that should be the numerator (FFLuc in NanoLuc, Venus in BRET)
# assyden - a string of the name of assay that should be the denominator (RLuc in NanoLuc)
# Returns a new dictionary with the {receptorname_assaynum/assayden : divided data} key value pairs.
def divplate(platedic, assaynum, assayden):
    divdict = {}
    plates = platedic.keys()
    for key in plates:
        receptor = key[:len(key) - len(assaynum)]
        receptorden = receptor + assayden
        if assaynum in key and receptorden in plates:
            divdict[f"{key}-{assayden}"] = platedic[key] / platedic[receptorden] 
    return divdict

#Averages n=1 and n=2 duplicates in a 384 well plate
#Accepts a dictionary of cleaned plates, and returns a dictionary of {plate name: averaged}
#and a dictionary of {plate name: standard deviation}
def average_plates(platestoaverage):
    averagedplates = {}
    stddev = {}
    for key, plate in platestoaverage.items():
        valuedf = pd.DataFrame()
        devdf = pd.DataFrame()
        for col in plate.columns:
            valuecol, devcol = average_stddev(plate[col])
            valuedf[col] = valuecol
            devdf[col] = devcol
        rename_averaged_axes(valuedf)
        rename_averaged_axes(devdf)
        averagedplates[key] = valuedf
        stddev[key] = devdf
    return averagedplates, stddev

def generate_abdict(ab_sheet):
    ab_dict = {}
    for index, series in ab_sheet.iterrows():
        
        start_molarity = series["Concentration (mg/mL)"] / series["MW"] / (10 * series["Dilution"])
        if start_molarity != 0:
            log_mol = math.log(start_molarity, 10)
        else:
            log_mol = 0
        conc_arr = np.arange(log_mol, log_mol - 8, -1)
        
        ab_dict[index] = (series["Name"], conc_arr)
    return ab_dict



def pivot_excel(plates_dict): 
    
    writer = pd.ExcelWriter("FFLuc-RLuc.xlsx", engine = "openpyxl")

    for key, plate in plates_dict.items():
        unstack = plate.unstack()
        unstack.to_excel(writer, sheet_name = key)
    writer.save()

#*** Obsolete by using multiindexing ***
#Accepts a dictionary of plates, and preps each plate for reshaping via .pivot
#Adds an "n" column, and collapses indeces to 8 (to account for duplicates)

# def prep_pivot(platedict):
#     for plate in platedict.values():
#         plate["n"] = pd.Series([2,1]*len(plate/2))
#         plate.rename_axis("Column", axis = "columns", inplace = True)
#         plate.rename_axis("Row", axis = "index", inplace = True)
#         plate.index = plate.index.map(collapse_dups)

#Mapper function to pass into Series.map()
#Used to collapse the indeces from 1->16 to 1->8 to account for duplicate rows in 384 well format
# def collapse_dups(index):
#     if index % 2 == 0:
#         return index / 2
#     return index // 2 + 1
        
#Takes the dataframe parsed from raw format off of M5 and changes Columns to be 1->24, and changes rows to be A->P
#Mutative function, doesn't return a new dataframe, but changes the existing dataframe that is passed in
def rename_axes(df):
    rownames = dict(zip(list(range(0, 16)), list(string.ascii_lowercase)[0:17]))
    colnames = dict(zip(list(range(2, 26)), list(range(1, 25))))
    df.rename(index = rownames, inplace=True)
    df.rename(columns = colnames, inplace = True)

#Changes rows from A->H
def rename_averaged_axes(df):
    rownames = dict(zip(list(range(0, 8)), list(string.ascii_lowercase)[0:8]))
    df.rename(index = rownames, inplace=True)

In [2]:
eln_uploader = widgets.FileUpload(multiple = False)
display("Upload ELN w/ Functional Data and Antibodies Sheet (xlsx), and specify assay type")
display(eln_uploader)

'Upload ELN w/ Functional Data and Antibodies Sheet (xlsx), and specify assay type'

FileUpload(value={}, description='Upload')

In [26]:
elnname = list(eln_uploader.value.keys())[0]
rawdata = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']))
headers = [i for i in range(len(rawdata.columns))]
headers[0] = rawdata.columns[0]
rawdata = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), names = headers)
ab_sheet = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), sheet_name = "Antibody Sheet")
ab_sheet.index += 1
#rawdata
ab_sheet

,Unnamed: 0.1,Unnamed: 0,IGGP Ticket#,BPP Ticket# (If available),Plate ID/Barcode,Well ID,Name,Transfection cell line,Purification Method,Concentration (mg/mL),Transfection scale (mL),Volume (uL),Yield (ug),Lot#,Date Delivered,Format Row,Format Column,MW,Dilution
1,NaN,NaN,NaN,NaN,NaN,NaN,C-Kit Ligand,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaT,NaN,NaN,19000,10
2,1.0,NaN,IGGP-509,BPP-XXX,pUBX_IGGP-509_221129_P001,A1,TB578_JMS_001,Hek-293,Phynexus,1.72,8.0,366.057,629.61804,PROT16713,2022-11-30,A,1.0,150000,1
3,2.0,NaN,NaN,NaN,NaN,B1,TB578_JMS_002,Hek-293,Phynexus,2.00,8.0,344.724,689.44800,PROT16714,2022-11-30,B,1.0,150000,1
4,3.0,NaN,NaN,NaN,NaN,C1,TB578_JMS_003,Hek-293,Phynexus,2.01,8.0,347.215,697.90215,PROT16715,2022-11-30,C,1.0,150000,1
5,4.0,NaN,NaN,NaN,NaN,D1,TB578_JMS_004,Hek-293,Phynexus,2.84,8.0,266.899,757.99316,PROT16716,2022-11-30,D,1.0,150000,1
6,5.0,NaN,NaN,NaN,NaN,E1,TB578_JMS_005,Hek-293,Phynexus,1.64,8.0,374.660,614.44240,PROT16717,2022-11-30,E,1.0,150000,1
7,6.0,NaN,NaN,NaN,NaN,F1,TB578_JMS_006,Hek-293,Phynexus,1.87,8.0,365.495,683.47565,PROT16718,2022-11-30,F,1.0,150000,1
8,7.0,NaN,NaN,NaN,NaN,G1,TB578_JMS_008,Hek-293,Phynexus,0.95,8.0,394.770,375.03150,PROT16719,2022-11-30,G,1.0,150000,1
9,8.0,NaN,NaN,NaN,NaN,H1,TB578_JMS_009,Hek-293,Phynexus,1.37,8.0,382.863,524.52231,PROT16720,2022-11-30,H,1.0,150000,1
10,9.0,NaN,NaN,NaN,NaN,A2,TB578_JMS_012,Hek-293,Phynexus,1.10,8.0,360.311,396.34210,PROT16721,2022-11-30,A,2.0,150000,1


In [1]:
#Returns a dictionary of series to correct concentrations of the passed in plate_dictionary {Name: Df} key value pairs
#Key of dictionary will be Antibody Name, Value will be the DataFrame with correct concentrations and n=1,2 pairs
def concentration_series(plate_dict, ab_dict):
    return_dict = {}
    for name, plate in plate_dict.items():
        
        for column_num in plate.columns:
            
            trueconc_arr = ab_dict[column_num][1]
            antibody_name = ab_dict[column_num][0]
            
            conc_mapper = dict(zip(np.arange(1,9), trueconc_arr))
            return_dict[antibody_name] = plate.loc[:, column_num].rename(index = conc_mapper, level = "Row").rename_axis(index = {"Row" : "Concentration log(M)"})
    return return_dict

numblocks = int(rawdata.columns[0][10:])

parsedplates = parse(rawdata, numblocks)
cleanplates = clean(parsedplates)
divideplates = divplate(cleanplates, "FFLuc", "RLuc")
individual_series = concentration_series(divideplates, generate_abdict(ab_sheet))
individual_series
pivot_excel(divideplates)

#averagedplates, stddevplates = average_plates(dividedplates)
#averagedplates

NameError: name 'rawdata' is not defined

In [57]:
for name, series in individual_series.items():
    series.plot(kind = 'scatter')

# tt = divideplates['CD117 SRF FFLuc-RLuc'].loc[:, 1]
# tt = tt.rename(index = concentration, level = "Row").rename_axis(index = {"Row" : "Concentration"})
# tt
#tt.plot(kind='scatter', x = "Concentration", y = 'n')

ValueError: plot kind scatter can only be used for data frames

In [ ]:
generator = ab_sheet.iterrows()
index, data = next(ab_sheet.iterrows())